# Covid Around The World

![Covid](../images/covid.jpeg)

## Install Widgets
ipwidgets is a library that allows you to interact with your data.

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Import Data Analysis Libraries
These are Python libraries specifically designed to manuplate  eg import, join datasets, analyise, filter, and clean data.

In [ ]:
import pandas as pd
import numpy as np

## Get Source Data
Data is sourced from a git hub repository for Johns Hopkins University COVID Data. The information is updated daily. The link is here:https://github.com/CSSEGISandData. <br>
<b>Note</b> Data is updated daily.


In [ ]:
#Recorded deaths since 1/1/2020
death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
#Confirmed Cases since 1/1/2020
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
#Recoveries since C since 1/1/2020
recovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
#Summanation of data by Country
country_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv", index_col=[0])

In [ ]:
# Check country data (Just type in the name of the dataframe and how many rows from the top(head))
country_df.head(3)

In [ ]:
# Check data has been imported correctly
confirmed_df.head(3)


In [ ]:
#Get the sape of the data
print (confirmed_df.shape)
print (recovered_df.shape)
print (death_df.shape)

## Data Cleansing
Data Cleansing is an important step in analysing data:
Here we change the column headings to lower case and simplify column heading names, and eliminate missing data. (Note normally you would not eliminate missing data without carrying out a missing data analysis)

In [ ]:
# Convert to Lowercase
#country_df
country_df.columns = map(str.lower,country_df.columns)
#confirmed_df
confirmed_df.columns = map(str.lower,confirmed_df.columns)
#recovered_df
recovered_df.columns = map(str.lower,recovered_df.columns)
#death_df
death_df.columns = map(str.lower,death_df.columns)


In [ ]:
# Check 
# country_df.head(2)
#confirmed_df.head(2)
recovered_df.head(2)
# death_df.head(2)

In [ ]:
# Rename columns to make it easier to refer to them
country_df = country_df.rename(columns={"country_region":"country","long_":"long"})
confirmed_df =confirmed_df.rename(columns = {'province/state':'state','country/region':'country'} )
recovered_df = recovered_df.rename(columns = {'province/state':'state','country/region':'country' } )
death_df =death_df.rename(columns = {'province/state':'state','country/region':'country' } )

In [ ]:
# displaying the datatypes and converting to integers
#display(country_df.dtypes) 
  
# converting 'Field_2' and 'Field_3' from float to int 
country_df['confirmed'] = country_df['confirmed'].apply(np.int64) 
country_df['deaths'] = country_df['deaths'].apply(np.int64) 
country_df['recovered'] = country_df['confirmed'].apply(np.int64) 
country_df['active'] = country_df['deaths'].apply(np.int64)   
# displaying the datatypes 
#display(country_df.dtypes) 

In [ ]:
# Recheck
country_df.head(2)
# confirmed_df.head(2)
# recovered_df.head(2)
# death_df.head(2)

## Sort Country Data 
Data is sorted in order of the number of confirmed cases. <br>
**Note** we are creating a new dataframe.

In [ ]:
sorted_country_df = country_df.sort_values('confirmed', ascending = False)

In [ ]:
sorted_country_df.head(10)

In [ ]:
# Create a function that will colour some of the columns for emphasis
def highlight_col(x):
    r = 'background-color: #7ED5EA'
    p = 'background-color: #63BCE5'
    g = 'background-color: #4B9FE1'
    temp_df = pd.DataFrame ('',index = x.index, columns = x.columns)
    temp_df.iloc[:,4]  = p
    temp_df.iloc[:,5]  = r
    temp_df.iloc[:,6]  = g
    return temp_df

In [ ]:
# Print the first 5 and style (calling the function)
sorted_country_df.head(5).style.apply(highlight_col, axis = None)

In [ ]:

sorted_country_df.style.format({
    'confirmed': '{:,.2f}'.format,
    'deaths': '{:,.2f}'.format,
    'recovered': '{:,.2%}'.format,
})
#sorted_country_df.head(3)

In [ ]:
# import ipywidgets as widgets
# from ipywidgets import interactive
 
# items = ['All']+sorted(sorted_country_df['country'].unique().tolist())
 
def view2(y=3):
    return sorted_country_df.head(y).style.apply(highlight_col, axis = None) 
a_slider = widgets.IntSlider(min=0, max=30, step=1, value=5)

widgets.interact(view2, y= a_slider)                           

## Import Data Visulation Library
There are many Python Graphical libraries that can be used. In this case we are going to use a library called Plotly. It can be installed using Pip Install or Conda install. It can be found @ https://plotly.com/python/ 

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(sorted_country_df.head(10), x = 'country', y = 'confirmed', size = 'confirmed',
                color = 'country', hover_name = 'country', size_max = 60)

In [ ]:
fig.show()

In [ ]:
import plotly.graph_objects as go
def plot_cases_for_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6,8]
    line_size = [4,6]
    df_list = [confirmed_df,death_df]
    fig = go.Figure()
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:,5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,5:]), axis=0)
        else:
            x_data = np.array(list(df.iloc[:,5:].columns))
            y_data = np.sum(np.asarray(df[df['country']==country].iloc[:,5:]), axis=0)
        fig.add_trace(go.Scatter(x=x_data, y= y_data, mode = 'lines+markers',
                                    name=labels[i],
                                    line = dict(color = colors[i], width = line_size[i]),
                                     connectgaps = True,
                                     text = "Total "+ str(labels[i])+ ": "+ str(y_data[-1])
                                               ))
    fig.show()

In [ ]:
# plot_cases_for_country('Ireland')
interact(plot_cases_for_country, country ='World');


## World Maps
Folium is a Python library used for visualizing geospatial data. In other words, Folium is a Python Library that enables you to import maps from around the world and allows you to represent data on them. For this excercise we will be using the Longitude(long) and Latitude(lat) to identify where our data comes from. Again you will have to install Folium using eithe Pip install or Conda install.

In [ ]:
import folium
confirmed_df=confirmed_df.dropna(subset=['long'])

confirmed_df=confirmed_df.dropna(subset=['lat'])

In [ ]:
world_map = folium.Map(location =[11,0],tiles = 'cartodbpositron', zoom_start = 2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map
                             
                             

In [ ]:
pd.set_option('display.max_rows', confirmed_df.shape[0]+1)
confirmed_df